# Optimal Strategies in RCV

We develop a computational framework to find optimal vote-addition strategies for each candidate -- if we want a candidate to place in the top $k$ of the election, what is the optimal (requiring the minimum number of additional ranked ballots) vote addition strategy? Our algorithmic framework is as follows (and as detailed in our paper), given $n$ candidates and $m$ unique voter ballots.

(1) Each set of ballots induces a structure, referring to the outcome order (main-structure) and the round-specific elimination or winning order (sub-structure). Of course, given an initial set of votes, one can reach any alternative structure by adding enough votes.  Given a budget of $B$ additional votes, we first develop an algorithm to optimally (if possible) reach a given structure in $O(mn)$ time. Then, a binary search on the budget $B$ yields the optimal strategy to efficiently reach a given structure. 

However, with many candidates, there are a prohibitively large number of structures: there are $n!$ possible orders (main structures), of which the candidate would be in the top $k$ in $k \times (n - 1)!$ main structures. Each main structure has $2^{n-1}$ sub-structures. Naively, then, finding an optimal strategy requires finding the minimum vote additions over $k \times (n - 1)! \times 2^{n-1}$ structures.  

(2) Thus, we develop approaches to reduce the election size, without affecting the optimality of the calculated strategies. (a) Given a budget of $B$ additional votes and status quo vote data, we next give an algorithm with $O(mn^4)$ complexity that removes a set of irrelevant candidates who will be eliminated first regardless of how the $B$ votes are added. (b) We show that for any set of $k$ winners, there are only $\sum_{j=1}^{k}$ $\mathcal{C}^n_k$ feasible substructures. Then, we show how to reduce the number of substructures further given status quo vote data. We give an algorithm with $O(mn^2)$ complexity that reduces the number of feasible sub-structures that can lead to an optimal win. 


## Preliminary functions

In [18]:
## all necessary packages

import pandas as pd
import numpy as np
import scipy
import random
import matplotlib.pyplot as plt
from itertools import combinations, groupby, permutations, product
import time 
from copy import deepcopy
import math
from collections import defaultdict, Counter
import seaborn as sns
from operator import itemgetter




### Basic utility functions

In [19]:
### Basic operations- generating voter data, doing permutations etc

#a function for checking if two lists have any common elements
def common_member(a, b): 
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(True)
    return(False)  

#given a list of candidates, this gives all possible orderings- n! in total
def main_structures(candidates):
    PotentialSets = [list(perm) for perm in permutations(candidates, len(candidates))]
    return PotentialSets

#returns a list of main strs for given list of winners
def str_for_given_winners(winners, candidates):
    losers = [item for item in candidates if item not in winners]
    PotentialSets=[]

    for perm_w in permutations(winners, len(winners)):
        for perm_l in permutations(losers, len(losers)):
            PotentialSets.append(list(perm_w)+list(perm_l))
    return PotentialSets

#returns a list of main strs for given list of losers
def str_for_given_losers(losers, candidates):
    winners = [item for item in candidates if item not in losers]
    PotentialSets=[]

    for perm_w in permutations(winners, len(winners)):
        for perm_l in permutations(losers, len(losers)):
            PotentialSets.append(list(perm_w)+list(perm_l))
    return PotentialSets
    
#returns a list of main strs for given list of winners and losers
def str_for_given_winners_losers(winners, candidates, losers):
    middle = [item for item in candidates if item not in winners or losers]
    PotentialSets=[]

    for perm_w in permutations(winners, len(winners)):
        for perm_l in permutations(losers, len(losers)):
            for perm_m in permutations(middle, len(middle)):
                PotentialSets.append(list(perm_w)+list(perm_m)+list(perm_l))
    return PotentialSets
    

#given a list of candidates, this gives all possible W/L round result types- 2^(n-1) in total
def sub_structures(candidates):
    l = len(candidates)-1
    results = list(product([0,1],repeat=l))
    roundresults = [list(ele)+[1] for ele in results]
    return roundresults

# Generate all possible combinations of 0 and 1 with z zeros at the beginning and (l - z) zeros in the middle
def sub_structures_specific(candidates, z):
    l = len(candidates) - 1
    
    prefix = [0] * z
    middle = list(product([0, 1], repeat=l - z))
    suffix = [1]
    
    # Combine the prefix, middle, and suffix to form the final combinations
    results = [prefix + list(ele) + suffix for ele in middle]
    
    return results

# given a main and sub structure, this gives a dict and a list showing who wins/loses in each round, in the order.
def create_structure(main, sub):
    maincopy = deepcopy(main)
    results = []
    redict = {} # mapping candidates to their results in the round order.
    for i in sub:
        if i == 0:
            res = maincopy.pop(-1)
            results.append([res, i])
        else:
            res = maincopy.pop(0)
            results.append([res, i])
        redict[res] = i     
    return results, redict

# given a list showing who wins/loses in each round, this gives the main and sub structure
def return_main_sub(results_list):
    strt = deepcopy(results_list)
    sub = [] # mapping candidates to their results in the round order.
    win=[]
    lose=[]
    for i in strt:
        if i[1]>0:
            win= win +[i[0]]
        else:
            lose= [i[0]] +lose
        sub.append(i[1])
    main = win+lose
    return main, sub


#while transferring from the last candidate to the current ones, we may pass those through checked candidates
#so here, we make permutations of checked candidates and add these paths between the transfers
def make_perms_transfer(candidates):
    perms_candidates = [] 
    for l in range(len(candidates)):
        perms = [list(perm) for perm in permutations(candidates, l)]
        perms_candidates = perms_candidates+perms
    return perms_candidates

# This is a general function for making all possible vote patterns of specified length, given the set of candidates. 
def make_perms(candidates, min_ballot_length=0):
    perms_candidates = [] 
    for l in range(min_ballot_length, len(candidates)):
        perms = [list(perm) for perm in permutations(candidates, l+1)]
        perms_candidates = perms_candidates+perms
    return perms_candidates

# initiates a dict of all possible winner types
def dict_perm_k(candidates, k):
    dict_new = {}
    for perm in permutations(candidates, k):
        dict_new[''.join(list(perm))] = Q
        
    return dict_new

# Returns the list of winners, given main str.
def give_winners(main_st, k):
    winners = main_st[0:k]
    return winners

In [20]:
### Basic dict operations- aggregation, non-aggregation, converting investment into complete aggr-data etc

# takes data in (type, num) format through lists and returns it in a dict format, but non-aggregated. 
def non_agg_dict(Ballottypes, NumVoters):
    stringballots = []
    for ballot in Ballottypes:
        ballotnew = ''.join(ballot)
        stringballots.append(ballotnew)

    non_aggre_voterdata_dict = dict(zip(stringballots, NumVoters))
    return non_aggre_voterdata_dict

#Take non-agg data and returns the complete aggr_dict.
#creates a dict with all V variables, where V_{AB} includes total votes that have first and second choices as A and B.
def agg_dict(candidates, non_aggre_voterdata):
    aggre_v_dict = Counter()
    permset = make_perms(candidates)

    # Calculate the aggregation using Counter
    for ballot in permset:
        for i in range(len(ballot)):
            v = ''.join(ballot[0:i+1])
            aggre_v_dict[v] += non_aggre_voterdata.get(''.join(ballot), 0)

    # Filter out keys with zero counts
    final_dict = {x: y for x, y in aggre_v_dict.items() if y > 0}
    return final_dict

# converts any investment or ballot counts into aggregated investment
def get_new_dict(my_dict):
    new_dict = {}
    
    # Iterate through the keys and values in my_dict
    for key, value in my_dict.items():
        # Generate all possible substrings of the key
        substrings = [key[:i+1] for i in range(len(key))]
        
        # Update the new_dict with the aggregated values for each substring
        for substring in substrings:
            new_dict[substring] = new_dict.get(substring, 0) + value
    
    # Filter out keys with zero counts
    final_dict = {x: y for x, y in new_dict.items() if y > 0}
    return final_dict


## Given the investment and the original aggr_dict, this returns the final aggr_dict.
def campaign_addition_dict(my_total_investment_dict, candidates, aggre_v_dict):
    
    my_total_investment_dict_d = deepcopy(my_total_investment_dict)
    aggre_v_dict_d = deepcopy(aggre_v_dict)
  
    agg_total_investment_dict = get_new_dict(my_total_investment_dict_d)

    for key in agg_total_investment_dict.keys():

        aggre_v_dict_d[key] = aggre_v_dict_d.get(key,0) + math.ceil(agg_total_investment_dict.get(key,0))
    
    return aggre_v_dict_d

## Reverse of aggregation operation. Given an aggregated dict, this gives a non-agg dict.
def clean_aggre_dict_diff(anydict):
    cleandict=deepcopy(anydict)
    for key1 in cleandict.keys():
        for key2 in cleandict.keys():
            if key2[0:len(key1)]==key1 and key1!=key2:
               
                cleandict[key1] = cleandict[key1]-cleandict[key2]
    final_dict = {x:y for x,y in cleandict.items() if y>0}
    return final_dict


## STV operations- navigating structures; finding optimal STV result

In [21]:
### Core functions of STV elections- coding up information in round-update, decoding later from their format

# After a round result (W/L) is reflected in checked_candidates, 
# this step updates the remaining vote banks by transfering the unused votes
def roundupdate(structure, checked_candidates, remaining_candidates, olddict):
    if len(checked_candidates)==0:
        return olddict
    currentdict = deepcopy(olddict)
    last_candidate = checked_candidates[-1] #depends on whether the last result is W or L, update changes
    
    perms_checked_candidates = make_perms_transfer(checked_candidates)
    
    stvotelist = currentdict[last_candidate] #last candidate's total vote bank
    for I_c in stvotelist:  #each set of votes that has a separate path to the last candidate
        I = deepcopy(I_c) #copying so that the Q updates don't change the original list accounts
        q = []
        while I[-1][0]==1: #we remove the lists with [1,..] from vote banks because these are fractions
            q1 = I.pop()
            q = q+[q1] #all removed fractions are stored here and appended later
        for j in remaining_candidates: # the vote banks of remaining candidates are updated here
            for l in perms_checked_candidates: #for each permutation of checked candidates that gets appended in middle
                if common_member(I, l) == False: #making sure the permutation candidates and the sets are distinct
                    if structure[last_candidate]==0: #last round is an elimination
                        Inew = I + l +[j]+ q 
                    else:    # last round is a win
                        Inew = I + l +[j] + q + [[1, 'Q', stvotelist]] #append a list for the surplus fraction
                    currentdict[j] =  currentdict[j]+[Inew]
    return currentdict


##Used to decode the previously coded descriptions in round_update. Gives dict containing numbers for current round.
# using aggregated data, converts each path into a number that measures the votes this path contributes
def decode_list(pathcopy, Q, aggre_v_dict):
    path = deepcopy(pathcopy)
    if path[-1][0]!=1:
        value =  aggre_v_dict.get(''.join(path), 0)  
        return value
    Qlist = []
    path2 = deepcopy(path)
    while path2[-1][0]==1:
        q_item = path2.pop()
        Qlist.append(q_item)
    tot = decode_list(path2, Q, aggre_v_dict)
    for q_item in Qlist:
        value = 0
        for part in q_item[-1]:
            value = value + decode_list(part, Q, aggre_v_dict)
        tot = tot *max(0, ( 1 - Q/(value+0.001)))
        #tot = tot *( 1 - Q/value)
    return tot

# using aggregated data, converts a dict with complete variable description to numerical data
def decode_dict(currentdict, candidates, Q, aggre_v_dict):
    newcurrentdict = {}
    for candidate in candidates:
        currentdict2 = deepcopy(currentdict)
        v_list = currentdict2[candidate]
        newcurrentdict[candidate] = round(sum(decode_list(path, Q, aggre_v_dict) for path in v_list),3)

    return newcurrentdict
        

In [22]:
### Produce optimal STV result ###


# creates a short dict from decoded_dict-for only ramining candidates- and returns the next elimination/win
# used in finding optimal STV result
def give_next_candidate(decoded_dict, remaining_candidates, Q, rt, dt):    
    short_decoded_dict = {can: decoded_dict[can] for can in remaining_candidates}
        
    # performs the next elimination/win and puts it in the checked_list
    t1 = max(short_decoded_dict, key=short_decoded_dict.get)
    t2 = min(short_decoded_dict, key=short_decoded_dict.get)
    if decoded_dict[t1]>=Q:
        t = t1
        rt.append([t, 1])
        dt[t] = 1
    else:
        t = t2
        rt.append([t, 0])
        dt[t] = 0
    return t, rt, dt


# for a given voter data, this produces the resulting optimal structure
# a bit similar to process_structure_STV
def STV_optimal_result(candidates, k, Q, aggre_v_dict):    

    currentdict  = {can: [[can]] for can in candidates}

    remaining_candidates = deepcopy(candidates)
    checked_candidates = []
    dt = {}
    rt = []
    
    for i in range(len(candidates)):
       
        # for i'th round, takes the currentdict and updates according to current round of elimination/win
        currentdict = roundupdate(dt, checked_candidates, remaining_candidates, currentdict)
        
        # once the variables are obtained for next round, this converts into numerical data
        decoded_dict = decode_dict(currentdict, candidates, Q, aggre_v_dict)
        
        t, rt, dt = give_next_candidate(decoded_dict, remaining_candidates, Q, rt, dt)
        
        remaining_candidates.remove(t) 
        checked_candidates.append(t)
        
    return rt, dt

In [30]:
##produce optimal social choice order for IRV or Single-winner RCV

def IRV_optimal_result(cands, ballot_counts):
    
    candsnew= deepcopy(cands)
    aggre_v_dict_mynew = get_new_dict(ballot_counts)
    results=[]
    
    for i in range(len(candsnew)):
        relevant_aggre_dict={}
    
        for c in candsnew:
        
            relevant_aggre_dict[c] =  aggre_v_dict_mynew.get(c, 0)
            
        worst_c = min(relevant_aggre_dict, key=relevant_aggre_dict.get)
        
        candsnew.remove(worst_c)
        results.insert(0, worst_c)
        # Initialize a dictionary for filtered data
        filtered_data = {}

        for key, value in ballot_counts.items():
            new_key = ''.join(char for char in key if char not in results)

            filtered_data[new_key] =   filtered_data.get(new_key, 0) + value
            
        filtered_data.pop('', None)
        aggre_v_dict_mynew={}

        aggre_v_dict_mynew = get_new_dict(filtered_data)
        
    return results
    

## Designing smart campaigns, holistically, for all rounds 

In [23]:

#  Returns the campaign investment needed for a round, given budget. Returns false, if this is not possible.
def add_campaign(log_campaign_list, main_st,  remaining_candidates, decoded_dict, Q, k, t, stdt, budget):
    current_campaign_dict = {j:0 for j in main_st} 
          
    if stdt[t]==1:
        v = []
        for candidate in remaining_candidates:
            v.append(decoded_dict[candidate])
        t_update = max(0,(max(v) - decoded_dict[t])+1, (Q - decoded_dict[t])+1)
        if t_update>budget:
            return {}, False
        current_campaign_dict[t] = t_update
    else: 
        for candidate in remaining_candidates:
            
            if decoded_dict[candidate] >= Q:
                
                return {}, False
            else:
                can_update = max(0, decoded_dict[t] - decoded_dict[candidate]+1)
                if can_update>budget:
                    return {}, False
                current_campaign_dict[candidate] = can_update 
            
    log_campaign_list.append(deepcopy(current_campaign_dict))

    return log_campaign_list, True

# Given a structure and voter data and an updated Q, this gives a round-wise campaign allocation requirement to 
# make the structure feasible. Provides fixed addition to be made in each round, by calling add_campaign.
# This function doesn't utilitze the budget to fulfil the structural requirements. Just lists those.
def process_campaign_STV(candidates, main, sub, k, Q, aggre_v_dict, budget):    
    
    strt, stdt = create_structure(main, sub)
    currentdict  = {can: [[can]] for can in candidates}

    remaining_candidates = list(stdt.keys())
    checked_candidates = []
    
    CheckDicts = [] # list of additions each round should have, with each round addition in dict format for candidates
    DecodedDicts = [] # list of how votes look according to given main and sub structure, in each round
    status_list = []
    log_campaign_list = []

    for i in range(len(candidates)-1):
        
        # for i'th round, takes the currentdict and updates according to current round of elimination/win
        currentdict = roundupdate(stdt, checked_candidates, remaining_candidates, currentdict)
        
        # once the variables are obtained for next round, this converts into numerical data
        decoded_dict = decode_dict(currentdict, candidates, Q, aggre_v_dict)
        # performs the next elimination/win and puts it in the checked_list
        t = remaining_candidates.pop(0) 
        checked_candidates.append(t)
        
        # checks if the next elimination/win is alright and if not, gives the addition that can make it right
        log_campaign_list, status = add_campaign(log_campaign_list, main, remaining_candidates, decoded_dict, Q, k, t, stdt, budget)
        if status==False:
            return {}, [], [], [False]
        status_list.append(status)
        DecodedDicts.append(decoded_dict)
        
    return DecodedDicts, strt, log_campaign_list, status_list

In [24]:
## This takes the round-wise requirement dict and returns the smart investment.
## This may not be very precise. Hence, we loop over this function later to final optimal investment.
## The problem arises when we invest in a losing candidate and latter rounds require investment in others to beat this
## candidate. Since we don't update log_campaign_dict while running this, we need loops outside the function.

def smart_campaign(candidates, log_campaign_list, strt, stdt, aggre_v_dict, Q, DecodedDicts, budget):
    copy_aggre_v_dict = deepcopy (aggre_v_dict)
    log = deepcopy(log_campaign_list)
    dict1 = log[0]  #for later distributing in the first round, where everyone gets what needed
    invest_dict = {can: 0 for can in candidates} #investment the candidates have in current rounds.
    total_investment_dict = {can: 0 for can in candidates} #total investment until this round
    
    #old investment in checked candidates, currently available for further dissipation 
    avl_dict = {can: 0 for can in candidates} 
    
    
    for can in dict1.keys(): #first round everyone gets what needed
        if dict1[can]>0:
            invest_dict[can] = dict1[can]
            total_investment_dict[can] = dict1[can]
             
    t = strt[0][0]      ## the first candidate for elimination or win
    decoded_dict = DecodedDicts[0]
    
    for rd in range(len(log)-1): # next rounds, with possible re-allocation of investment for efficiency 
        
        #first update the log of checked candidates. Dissipate investment into available dict.
        if stdt[t] == 1 and invest_dict[t]>0:
            dee_invest = deepcopy(invest_dict)
            avl_dict[t] = math.floor((decoded_dict[t]+ dee_invest[t]-Q)*(dee_invest[t]/(decoded_dict[t]+dee_invest[t])))
            invest_dict[t] = 0
            
        if stdt[t] == 0 and invest_dict[t]>0:
            for rich_can in total_investment_dict.keys():
                if rich_can[len(rich_can)-1]==t: #for all investments that are currently owned by t
                    avl_dict[rich_can] = deepcopy(total_investment_dict)[rich_can]
                
            invest_dict[t] = 0
        
        # Then, process the next checking of candidates.
        dict_rd = log[rd+1] #current round requirement 
        decoded_dict = DecodedDicts[rd+1] #this is needed for the updated votes info
        for can in dict_rd.keys():
            if dict_rd[can]>invest_dict[can]:
                needed_extra =  dict_rd[can] - invest_dict[can]
                invest_dict[can] = dict_rd[can] #investment currently the candidate has
                for rich_can in avl_dict.keys(): #check if old checked candidates can help
                    if avl_dict[rich_can]>0:  
                        rich_can_gives = min(avl_dict[rich_can], needed_extra)
                        avl_dict[rich_can] = avl_dict[rich_can] - rich_can_gives
                        total_investment_dict[rich_can] = total_investment_dict[rich_can]-rich_can_gives
                        total_investment_dict[str(rich_can)+str(can)] = rich_can_gives # ballots with >1 choices.
                        needed_extra = needed_extra - rich_can_gives
                if needed_extra>0:
                    total_investment_dict[can] = total_investment_dict[can]+needed_extra
        t = strt[rd+1][0]   
        amount_spent = sum(total_investment_dict[key] for key in total_investment_dict.keys())   
        if amount_spent >budget:
            return {}, amount_spent
                    
    return total_investment_dict, amount_spent


## Implementing smart campaign functions to optimize over structures

In [25]:
### Given main and sub str specifications, this function gives the updated dict and amount that does the job.
### Uses the smart campaign function to find optimal investment dict and uses addition to give final dict.
### Uses the while loop to repeatedly do the additions, until the given str becomes optimal. 

def reach_a_structure_check(candidates, main, sub, k, Q_new, aggre_v_dict, budget):
    amount_check = 1
    check_aggre_v_dict = deepcopy(aggre_v_dict)
    
    strt, stdt = create_structure(main, sub)
    
    while amount_check > 0: #until the additional amount that needs to be spent becomes zero
        
        DecodedDicts, strt, log_campaign_list, status_list = process_campaign_STV(candidates, main, sub, k, Q_new, check_aggre_v_dict, budget)
        
        if all(status_list)==True: # proceed only if campaigning is feasible. 
            total_investment_dict, amount_check = smart_campaign(candidates, log_campaign_list, strt, stdt, check_aggre_v_dict, Q_new, DecodedDicts, budget)
            check_aggre_v_dict = campaign_addition_dict(total_investment_dict, candidates, check_aggre_v_dict)
            if amount_check>budget:
                return False, {}, 0
      
        else:
            return False, {}, 0
    #total amount spent in the process     
    amount_spent = sum(check_aggre_v_dict[candidate] for candidate in candidates) - sum(aggre_v_dict[candidate] for candidate in candidates)
    
    return amount_spent<budget, check_aggre_v_dict, amount_spent

In [26]:
### This function uses the 'reach_a_structure_check' function and goes over all sub structures to find the minimum amount
### to reach the flipped order. Can be generalized to reach any desired main str. 

def flip_order_campaign(candidates, k, Q, aggre_v_dict, budget):
    
    strt_og, stdt_og = STV_optimal_result(candidates, k, Q, aggre_v_dict)
    original_main, original_sub = return_main_sub(strt_og)
    Q_new = Q+budget/(k+1)
    budget_list_flip= []
    campaigned_dict_list = []
    for sub in sub_structures(candidates):

        status_final, check_aggre_v_dict, amount_spent = reach_a_structure_check(candidates, list(reversed(original_main)), sub, k, Q_new, aggre_v_dict, budget)
        
        if status_final==True:
            
            campaigned_dict_list.append(check_aggre_v_dict)
            budget_list_flip.append(amount_spent)
    
    if len(budget_list_flip)==0:
        print('increase the budget')
        return {}, 0, {}
    else:        
        min_budget = min(budget_list_flip)
        print(budget_list_flip)
        min_index=budget_list_flip.index(min_budget)

        check_aggre_v_dict =  campaigned_dict_list[min_index]
        strt_new, stdt_new = STV_optimal_result(candidates, 2, Q_new, check_aggre_v_dict)
        new_main, new_sub = return_main_sub(strt_new)
        C = {x: check_aggre_v_dict[x] - aggre_v_dict[x] for x in check_aggre_v_dict if x in aggre_v_dict}
        diff = {x:y for x,y in C.items() if y!=0}
        
        print('New votes to be added = ', clean_aggre_dict_diff(diff))
        print('original order = ', original_main, original_sub)
        print('new order = ', new_main, new_sub)
        print('budget used = ', min_budget)
        return check_aggre_v_dict, Q_new, clean_aggre_dict_diff(diff)


In [27]:
## This function finds minimum additions required for each combination of top k positions, i.e., winners

def reach_any_winners_campaign(candidates, k, Q, aggre_v_dict, budget):
    strt_og, stdt_og = STV_optimal_result(candidates, k, Q, aggre_v_dict)
    original_main, original_sub = return_main_sub(strt_og)
    Q_new = Q+budget/(k+1)
    
    og_winners = give_winners(original_main, k)
    strats_frame = {}
    strats_frame[''.join(og_winners)] = [0, []]
    for comb in combinations(candidates, k):
      
      if set(comb)!=set(og_winners):  
        main_set = str_for_given_winners(comb, candidates)
        #strats_frame[''.join(comb)] = [-Q_new, []]
    
        for current_main in main_set:
            budget_list_flip= []
            campaigned_dict_list = []
            for sub in sub_structures(candidates): #sub_structures(candidates)

                status_final, check_aggre_v_dict, amount_spent = reach_a_structure_check(candidates, current_main, sub, k, Q_new, aggre_v_dict, budget)


                if status_final==True:

                    campaigned_dict_list.append(check_aggre_v_dict)
                    budget_list_flip.append(amount_spent)
        
                    

            if len(budget_list_flip)>0:       
                min_budget = min(budget_list_flip)
              
                min_index=budget_list_flip.index(min_budget)

                check_aggre_v_dict =  campaigned_dict_list[min_index]
                
                strt_new, stdt_new = STV_optimal_result(candidates, k, Q_new, check_aggre_v_dict)
                new_main, new_sub = return_main_sub(strt_new)
                if set(give_winners(new_main, k))!=set(comb):
                    print('error!')
                C = {x: check_aggre_v_dict[x] - aggre_v_dict[x] for x in check_aggre_v_dict if x in aggre_v_dict}
                diff = {x:y for x,y in C.items() if y>0}
                if strats_frame.get(''.join(comb), [budget, {}])[0] > min_budget:
        
                    strats_frame[''.join(comb)] = [min_budget, clean_aggre_dict_diff(diff)]
           
    return {x:y for x,y in strats_frame.items() if y[0]>=0}

In [28]:
## This function finds minimum additions needed to reach any structure. Produces a dict of all reachable orders 
# within the specified budget.

def reach_any_order_campaign(candidates, k, Q, aggre_v_dict, budget):
    strt_og, stdt_og = STV_optimal_result(candidates, k, Q, aggre_v_dict)
    original_main, original_sub = return_main_sub(strt_og)
    Q_new = Q+budget/(k+1)
    
    og_winners = give_winners(original_main, k)
    strats_frame = {}
    strats_frame[''.join(og_winners)] = [0, []]
    
    main_set = main_structures(candidates)
    
    for current_main in main_set:
        budget_list_flip= []
        campaigned_dict_list = []
        for sub in sub_structures(candidates): #sub_structures(candidates)

            status_final, check_aggre_v_dict, amount_spent = reach_a_structure_check(candidates, current_main, sub, k, Q_new, aggre_v_dict, budget)


            if status_final==True:

                campaigned_dict_list.append(check_aggre_v_dict)
                budget_list_flip.append(amount_spent)



        if len(budget_list_flip)>0:       
            min_budget = min(budget_list_flip)

            min_index=budget_list_flip.index(min_budget)

            check_aggre_v_dict =  campaigned_dict_list[min_index]

            strt_new, stdt_new = STV_optimal_result(candidates, k, Q_new, check_aggre_v_dict)
            new_main, new_sub = return_main_sub(strt_new)
           
            C = {x: check_aggre_v_dict[x] - aggre_v_dict[x] for x in check_aggre_v_dict if x in aggre_v_dict}
            diff = {x:y for x,y in C.items() if y>0}
            if strats_frame.get(''.join(new_main), [budget, {}])[0] > min_budget:

                strats_frame[''.join(new_main)] = [min_budget, clean_aggre_dict_diff(diff)]
           
    return {x:y for x,y in strats_frame.items() if y[0]>=0}

# Removal of candidates under uncertainty

In [29]:
## To check if we can remove group, and retain candidates

def check_removal(candidates, group, ballot_counts, budget):
    #strict support check
    #group = 'FGHIJKLM'

    strict_support = {}

    for key, value in ballot_counts.items():
        i = 0
        newset = []
        while key[i] in group:
            newset.append(key[i])
            i=i+1
        new_key = ''.join(char for char in newset)
        for letter in new_key:
            if letter in strict_support:
                strict_support[letter] += value
            else:
                strict_support[letter] = value
    can_remove = False
   
    for best_c_irrelevant in strict_support.keys():

        #best_c_irrelevant = max(strict_support, key=strict_support.get)
        groupcopy = group

        mostly_irrelevant = groupcopy.replace(best_c_irrelevant, "")

        # Initialize a dictionary for filtered data
        filtered_data = {}

        # Remove letters maybe_irrelevant {G, H, I, J, K, L}? while retaining the rest of the string
        for key, value in ballot_counts.items():
            new_key = ''.join(char for char in key if char not in mostly_irrelevant)

            filtered_data[new_key] =   filtered_data.get(new_key, 0) + value
        filtered_data.pop('', None)

        aggre_v_dict = get_new_dict(filtered_data)

        relevant_aggre_dict = {}
        #candidates = ['A', 'B', 'C', 'D', 'E']

        for c in candidates:
            relevant_aggre_dict[c] =  aggre_v_dict[c]

        worst_c_relevant = min(relevant_aggre_dict, key=relevant_aggre_dict.get)
        #print(int(relevant_aggre_dict[worst_c_relevant]-strict_support[best_c_irrelevant]))
        if int(relevant_aggre_dict[worst_c_relevant]-strict_support[best_c_irrelevant])+1>= budget:
            can_remove = True
        else: #again check if the addition really changes who drops out after E removal
            last_three= sorted(relevant_aggre_dict.items(), key=itemgetter(1))[:3] 
            #budget not enough to benefit 2 candidates at the bottom
            
            if 2*last_three[2][1]-last_three[1][1]-last_three[0][1] >budget:
            
                # Remove letters maybe_irrelevant {E, G, H, I, J, K, L}? while retaining the rest of the string

                maybe_irrelevant = mostly_irrelevant+ worst_c_relevant
                    # Initialize a dictionary for filtered data
                filtered_data = {}

                for key, value in ballot_counts.items():
                    new_key = ''.join(char for char in key if char not in maybe_irrelevant)

                    filtered_data[new_key] =   filtered_data.get(new_key, 0) + value
                filtered_data.pop('', None)

                aggre_v_dict = get_new_dict(filtered_data)

                #add budget votes to F
                aggre_v_dict[best_c_irrelevant] = aggre_v_dict[best_c_irrelevant]+ budget

                # new candidate list that removes E and adds F
                relevant_aggre_dict = {}
                candidates_temp = candidates.copy()
                candidates_temp.remove(worst_c_relevant)
                candidates_temp.append(best_c_irrelevant)

                for c in candidates_temp:
                    relevant_aggre_dict[c] =  aggre_v_dict[c]

                new_best_c_irrelevant = min(relevant_aggre_dict, key=relevant_aggre_dict.get)


                if new_best_c_irrelevant == best_c_irrelevant: #F still gets eliminated
                    can_remove = True
                    #return True
                else:
                    #print(new_best_c_irrelevant)
                    can_remove = False
                    return False
                    #return False
    return can_remove

def remove_irrelevent( ballot_counts, cands_dontwant, startcandidates, budget, fullgroup, stop):

    candidatesnew = startcandidates
    #candidatesnew = candidatesnew[:6]
    group = ''.join(char for char in fullgroup if char not in candidatesnew)
   
    while check_removal(candidatesnew, group, ballot_counts, budget)!=True:
        candidatesnew = candidatesnew[:-1]
        #print(candidatesnew)
        group = ''.join(char for char in fullgroup if char not in candidatesnew)
        if len(candidatesnew)<=2:
            stop =True
            break
    return candidatesnew, group, stop
       


# Primary elections experiments

## The poll data

In [31]:
# Create a mapping from candidate names to variables
candidates_mapping = {
    "Trump": "A",
    "Haley": "B",
    "Ramaswamy": "C",
    "DeSantis": "D",
    "Christie":"E",
    "Pence": "F",
    "Scott": "G",
    "Hurd": "H",
    "Hutchinson": "I",
    "Youngkin": "J",
    "Burgum": "K",
    "Elder": "L",
    "Suarez": "M",
}


In [32]:
## Case studies: republican primary

df = pd.read_csv("republican_primary/poll_data.csv")
df = df.drop(columns = ['record', 'QBASE'])
df.head()


# Initialize a dictionary to store the counts of each ballot type as strings
ballot_counts = {}

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    # Initialize an empty list to store valid candidates
    valid_candidates = []
    
    # Iterate through columns rank1 to rank5
    for i in range(1, 14):
        candidate = row[f'rank{i}']
        valid_candidates.append(candidates_mapping[candidate])
    

    ballot_type = ''.join(valid_candidates)

    # Use the 'weight' column to determine the number of voters for this ballot type
    weight = row['weight']

    # Add the weight to the count for this ballot type in the dictionary
    if ballot_type not in ballot_counts:
        ballot_counts[ballot_type] = weight
    else:
        ballot_counts[ballot_type] += weight

print(len(ballot_counts))
#ballot_counts['E'] = 30
# ballot_counts['D'] = 7

801


In [45]:
# Initialize a dictionary for filtered data
filtered_data = {}

# Remove letters {G, H, I, J, K, L} while retaining the rest of the string
for key, value in ballot_counts.items():
    new_key = ''.join(char for char in key if char not in 'FGHIJKLM')
    
    filtered_data[new_key] =   filtered_data.get(new_key, 0) + value
filtered_data.pop('', None)
print(len(filtered_data))
# aggre_v_dict = get_new_dict(filtered_data)
# candidates = ['A', 'B', 'C', 'D']
# for c in candidates:
#     print(c, aggre_v_dict[c])

100


In [46]:
candidates = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
candidates = ['A', 'B', 'C', 'D', 'E']#, 'C', 'D', 'E', 'F', 'G']
#filtered_data  = ballot_counts
#aggre_v_dict = get_new_dict(ballot_counts)
aggre_v_dict = get_new_dict(filtered_data)
#print(aggre_v_dict)
for c in candidates:
    print(c, aggre_v_dict[c])

#print(aggre_v_dict)
k=1
Q = round(sum(aggre_v_dict[candidate] for candidate in candidates)/(k+1)+1,3)+400
print(Q)

#strict support check
group = 'BGHIJKLM'

letter_counts = {}

#strict_support
for key, value in ballot_counts.items():
    i = 0
    newset = []
    while key[i] in group:
        newset.append(key[i])
        i=i+1
    new_key = ''.join(char for char in newset)
    for letter in new_key:
        if letter in letter_counts:
            letter_counts[letter] += value
        else:
            letter_counts[letter] = value

print(letter_counts)


best_c_irrelevant = max(letter_counts, key=letter_counts.get)


#if int(aggre_v_dict['E']-letter_counts[best_c_irrelevant])>=40:
#    print('delete them')
    


A 387.9737379999999
B 95.082114
C 118.92247600000002
D 112.76345800000001
E 86.25825999999999
801.5
{'B': 82.14700099999996, 'K': 15.527009, 'G': 42.271861, 'I': 20.949628999999998, 'L': 8.297495, 'H': 17.005557, 'J': 15.029082, 'M': 10.800882}


In [47]:
for c in ['EA','EC', 'ED']:
    #print(c, aggre_v_dict[c]/aggre_v_dict['E']*8.84)
    #print(c, aggre_v_dict[c]/aggre_v_dict['E']*8.84)
    aggre_v_dict[c] = aggre_v_dict[c]+aggre_v_dict[c]/aggre_v_dict['E']*8.84
    print(aggre_v_dict[c])

1.9338884754858259
8.466721689587757
10.160136586413405


In [48]:
aggre_v_dict['E'] = aggre_v_dict['E']+ 2

rt, dt = STV_optimal_result(candidates, k, Q, aggre_v_dict)

In [49]:
print(rt, dt)

[['E', 0], ['D', 0], ['C', 0], ['B', 0], ['A', 0]] {'E': 0, 'D': 0, 'C': 0, 'B': 0, 'A': 0}


In [50]:
start = time.time()
#strats_frame = reach_any_winners_campaign(candidates, 2, Q, aggre_v_dict, 300)
strats_frame = reach_any_order_campaign(candidates, 2, Q, aggre_v_dict, 40)
end = time.time()
print('total time = ' , end - start)
print( strats_frame)

total time =  0.9499368667602539
{'AB': [0, []], 'ABCDE': [0.0, {}], 'ABDCE': [6.0, {'D': 6.0}], 'ACBDE': [33.0, {'C': 33.0}], 'ACDEB': [11.0, {'E': 8.0, 'D': 3.0}], 'ACEDB': [8.0, {'E': 8.0}], 'ADCEB': [26.0, {'D': 18.0, 'ED': 8.0}], 'ADECB': [21.0, {'E': 14.0, 'D': 7.0}], 'AEDCB': [35.0, {'E': 28.0, 'D': 7.0}]}


In [51]:
strats_frame2 = reach_any_winners_campaign(candidates, 2, Q, aggre_v_dict, 40)
print( strats_frame2)

{'AB': [0, []], 'AC': [8.0, {'E': 8.0}], 'AD': [21.0, {'E': 14.0, 'D': 7.0}], 'AE': [35.0, {'E': 28.0, 'D': 7.0}]}


## Optimal Strategies under uncertainty

In [ ]:
#ballot_counts['E'] = 10
import os
from collections import Counter


start = time.time()

bootstrap_samples_dir = 'bootstrap_samples_new_3'

# List all files in the directory
bootstrap_files = os.listdir(bootstrap_samples_dir)

strats_under_uncertainty = []

data_samples = []
# Loop over each file and load it as a DataFrame
for file in bootstrap_files:
    stop = False
    it=it+1
    sample_file_path = os.path.join(bootstrap_samples_dir, file)
    
    df = pd.read_csv(sample_file_path)
    
    #df = df.drop(columns = ['record', 'QBASE'])

    # Initialize a dictionary to store the counts of each ballot type as strings
    ballot_counts = {}
    Q = 800

    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        # Initialize an empty list to store valid candidates
        valid_candidates = []

        # Iterate through columns rank1 to rank5
        for i in range(1, 14):
            candidate = row[f'rank{i}']
            valid_candidates.append(candidates_mapping[candidate])


        ballot_type = ''.join(valid_candidates)

        # Use the 'weight' column to determine the number of voters for this ballot type
        weight = row['weight']

        # Add the weight to the count for this ballot type in the dictionary
        if ballot_type not in ballot_counts:
            ballot_counts[ballot_type] = weight
        else:
            ballot_counts[ballot_type] += weight
            
    df['weight'] = df['weight']*800/df['weight'].sum()
    #ballot_counts['EC'] = 10
    
    
    
    results = IRV_optimal_result(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'], ballot_counts)
    strats_under_uncertainty. append(''.join(results[:2]))

    
end = time.time()    
print('total time = ' , end - start)

frequency = Counter(strats_under_uncertainty)

# Displaying the frequency of each element
print(frequency)

In [ ]:
import numpy as np

def generate_bootstrap_samples(data, n_samples=1000):
    """
    Generates bootstrap samples from the given dataset.

    Parameters:
    data (DataFrame): The original dataset containing RCV rankings.
    n_samples (int): The number of bootstrap samples to generate.

    Returns:
    List[DataFrame]: A list containing the bootstrap samples.
    """
    bootstrap_samples = []
    n = len(data)
    for _ in range(n_samples):
        sample = data.sample(n, replace=True)  # Sampling with replacement
        bootstrap_samples.append(sample)
    return bootstrap_samples


n_bootstrap_samples = 1000  # You can change this to your desired number of samples
bootstrap_samples = generate_bootstrap_samples(df, n_bootstrap_samples)


import os

# Creating a directory to store all the bootstrap sample files
output_dir = 'bootstrap_samples_new'
os.makedirs(output_dir, exist_ok=True)

# Saving each bootstrap sample as a separate CSV file
for i, sample in enumerate(bootstrap_samples):
    sample_file_path = os.path.join(output_dir, f'bootstrap_sample_{i+1}.csv')
    sample.to_csv(sample_file_path, index=False)

# Creating a ZIP file of all the bootstrap sample files
import shutil
zip_file_path = 'bootstrap_samples_new.zip'
shutil.make_archive(base_name=os.path.splitext(zip_file_path)[0], format='zip', root_dir=output_dir)

zip_file_path

In [ ]:
import os



# final: bootstrap_samples_dir = 'bootstrap_samples_new_3'

bootstrap_samples_dir = 'bootstrap_samples_new_3'

# List all files in the directory
bootstrap_files = os.listdir(bootstrap_samples_dir)
it=0
budget = 40

algo_works = 0

data_samples = []
start = time.time()
# Loop over each file and load it as a DataFrame
for file in bootstrap_files:
    
    stop = False
    it=it+1
    sample_file_path = os.path.join(bootstrap_samples_dir, file)
    
    df = pd.read_csv(sample_file_path)
    
    #df = df.drop(columns = ['record', 'QBASE'])

    # Initialize a dictionary to store the counts of each ballot type as strings
    ballot_counts = {}
    Q = 800

    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        # Initialize an empty list to store valid candidates
        valid_candidates = []

        # Iterate through columns rank1 to rank5
        for i in range(1, 14):
            candidate = row[f'rank{i}']
            valid_candidates.append(candidates_mapping[candidate])


        ballot_type = ''.join(valid_candidates)

        # Use the 'weight' column to determine the number of voters for this ballot type
        weight = row['weight']

        # Add the weight to the count for this ballot type in the dictionary
        if ballot_type not in ballot_counts:
            ballot_counts[ballot_type] = weight
        else:
            ballot_counts[ballot_type] += weight
            
    #df['weight'] = df['weight']*800/df['weight'].sum()
    
    results = IRV_optimal_result(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'], ballot_counts)
    #print(results)
    
    candidates, group, stop = remove_irrelevent( ballot_counts, results[-7:], 
                results[:6], budget, 'ABCDEFGHIJKLM', stop)
    
    if stop==False:
        
        algo_works = algo_works+1

        # Initialize a dictionary for filtered data
        filtered_data = {}

        # Remove letters {G, H, I, J, K, L} while retaining the rest of the string
        for key, value in ballot_counts.items():
            new_key = ''.join(char for char in key if char not in group)

            filtered_data[new_key] =   filtered_data.get(new_key, 0) + value
        filtered_data.pop('', None)
        aggre_v_dict_new = get_new_dict(filtered_data)


        k=1
        Q = round(sum(aggre_v_dict_new [candidate] for candidate in candidates)/(k+1)+1,3)+400

        strats_frame = reach_any_winners_campaign(candidates, 2, Q, aggre_v_dict_new , budget)
        data_samples.append(strats_frame)
        
    
end = time.time()    
print('total time = ' , end - start)
print(algo_works, it)
print(strats_frame)

In [ ]:
from collections import defaultdict

# Initialize dictionaries for analysis
winning_combinations_frequency = defaultdict(int)
additional_votes_needed = defaultdict(lambda: defaultdict(int))
vote_addition_frequency = defaultdict(lambda: defaultdict(int))

# Analyze each sample
for sample in data_samples:
    for combination, (value, details) in sample.items():
        # Winning combinations
        if value == 0:
            winning_combinations_frequency[combination] += 1
        else:
            # Additional votes needed
            additional_votes_needed[combination][value] += 1
            if isinstance(details, dict):
                for candidate, votes in details.items():
                    # Frequency of vote addition to each candidate
                    vote_addition_frequency[combination][candidate] += 1

# Convert defaultdict to regular dict for cleaner output
winning_combinations_frequency = dict(winning_combinations_frequency)
additional_votes_needed = {k: dict(v) for k, v in additional_votes_needed.items()}
vote_addition_frequency = {k: dict(v) for k, v in vote_addition_frequency.items()}

winning_combinations_frequency, additional_votes_needed, vote_addition_frequency


In [ ]:
from collections import defaultdict
from itertools import combinations

# Initialize dictionary for detailed vote addition frequency
detailed_vote_addition_frequency = defaultdict(lambda: defaultdict(int))

# Analyze each sample in data_samples
for sample in data_samples:
    for combination, (value, details) in sample.items():
        if value > 0 and isinstance(details, dict):
            # Count single candidate additions
            for candidate in details:
                if len(candidate) == 1:  # Single candidate
                    detailed_vote_addition_frequency[combination][f"Single-ranked: {candidate}"] += 1
                else:  # Multi-ranked
                    detailed_vote_addition_frequency[combination][f"Multi-ranked: {candidate}"] += 1

            # Count combinations of separate additions
            if len(details) > 1:
                separate_candidates = [candidate for candidate in details if len(candidate) == 1]
                for num in range(2, len(separate_candidates) + 1):
                    for combo in combinations(separate_candidates, num):
                        combo_key = " & ".join(sorted(combo))
                        detailed_vote_addition_frequency[combination][f"Combination: {combo_key}"] += 1

# Convert defaultdict to regular dict for cleaner output
formatted_detailed_vote_addition_frequency = {k: dict(v) for k, v in detailed_vote_addition_frequency.items()}

formatted_detailed_vote_addition_frequency



In [ ]:
# Initialize a counter to track the frequency of candidates winning with 0 additions
winning_frequency = Counter()

# Iterate through each entry in the data
for entry in data_samples:
    for candidate, values in entry.items():
        # Check if the additional votes required is 0
        if values[0] == 0:
            # Increment the count for this candidate
            winning_frequency[candidate] += 1

winning_frequency


In [ ]:
# Initialize dictionaries to track the total additions and the count of instances for each combination
total_additions = Counter()
win_with_additions = Counter()

# Iterate through each entry in the data
for entry in data_samples:
    for candidate, values in entry.items():
        additions, details = values
        # Check if the combination is not winning (additions not 0)
        if additions != 0:
            # Sum the additions made for each candidate within the combination
            total_additions_for_candidate = sum(details.values())
            # Update the total additions and count for this combination
            total_additions[candidate] += total_additions_for_candidate
            win_with_additions[candidate] += 1

# Calculate the average additions for each combination
average_additions = {candidate: total_additions[candidate] / win_with_additions[candidate] 
                     for candidate in total_additions}

win_hook_or_crook  = {candidate: round((win_with_additions[candidate]+winning_frequency[candidate])
                                       /algo_works*100,3) for candidate in win_with_additions}

print('average additions =',average_additions)
print('win using additions =', win_with_additions)
print('Winning frequncy = ', winning_frequency)
print('win by hook or crook for moe', budget/8, '=',  win_hook_or_crook)

In [ ]:
# Reinitialize the dictionary to track the frequencies of each unique type of addition for each combination
addition_frequencies = defaultdict(Counter)

# Function to extract individual candidates from the addition types
def extract_candidates(addition_types):
    candidates = set()
    for addition in addition_types:
        for candidate in addition:
            candidates.add(candidate)
    return ''.join(sorted(candidates))

# Iterate through each entry in the data
for entry in data_samples:
    for candidate, values in entry.items():
        additions, details = values
        # Check if the combination is not winning (additions not 0)
        if additions != 0:
            # Extract individual candidates from the addition types
            addition_category = extract_candidates(details.keys())
            # Increment the frequency for this category of addition
            addition_frequencies[candidate][addition_category] += 1

dict(addition_frequencies)  # Convert defaultdict to regular dict for better readability



In [ ]:
import matplotlib.pyplot as plt 
# Function to calculate the average non-zero deviations for each dictionary
def average_non_zero_deviations(data_samples):
    averages = []

    for dict_item in data_samples:
        total = 0
        count = 0
        for key, value in dict_item.items():
            deviation = value[0]
            if deviation != 0:
                total += deviation
                count += 1
        average = total / count if count != 0 else 0
        averages.append(average)

    return averages

# Calculate the averages
average_deviations = average_non_zero_deviations(data_samples)
average_deviations.count(0)
print(len(average_deviations))
plt.hist(average_deviations)